Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import sklearn.ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import OrdinalEncoder
from lightgbm import LGBMRegressor
from sklearn.linear_model import LassoCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.utils import resample
import statsmodels
from statsmodels.tsa.statespace.sarimax import SARIMAX
import shap

Importing datasets

In [2]:
data1 = pd.read_csv('C:/KUSHAL/Projects/Kaggle/Rohlik Sales Forecasting/inbox/sales_train.csv', encoding = 'utf-8')
data2 = pd.read_csv('C:/KUSHAL/Projects/Kaggle/Rohlik Sales Forecasting/inbox/sales_test.csv', encoding = 'utf-8')
data3 = pd.read_csv('C:/KUSHAL/Projects/Kaggle/Rohlik Sales Forecasting/inbox/calendar.csv', encoding = 'utf-8')
data4 = pd.read_csv('C:/KUSHAL/Projects/Kaggle/Rohlik Sales Forecasting/inbox/inventory.csv', encoding = 'utf-8')
data5 = pd.read_csv('C:/KUSHAL/Projects/Kaggle/Rohlik Sales Forecasting/inbox/solution.csv', encoding = 'utf-8')
data6 = pd.read_csv('C:/KUSHAL/Projects/Kaggle/Rohlik Sales Forecasting/inbox/test_weights.csv', encoding = 'utf-8')
print("Datasets successfully imported")

Datasets successfully imported


In [3]:
print(data1.info())
print(data1.isnull().sum()) #info of train set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4007419 entries, 0 to 4007418
Data columns (total 14 columns):
 #   Column           Dtype  
---  ------           -----  
 0   unique_id        int64  
 1   date             object 
 2   warehouse        object 
 3   total_orders     float64
 4   sales            float64
 5   sell_price_main  float64
 6   availability     float64
 7   type_0_discount  float64
 8   type_1_discount  float64
 9   type_2_discount  float64
 10  type_3_discount  float64
 11  type_4_discount  float64
 12  type_5_discount  float64
 13  type_6_discount  float64
dtypes: float64(11), int64(1), object(2)
memory usage: 428.0+ MB
None
unique_id           0
date                0
warehouse           0
total_orders       52
sales              52
sell_price_main     0
availability        0
type_0_discount     0
type_1_discount     0
type_2_discount     0
type_3_discount     0
type_4_discount     0
type_5_discount     0
type_6_discount     0
dtype: int64


In [4]:
pd.set_option('display.max_rows', None)
null_count = data1[data1['total_orders'].isnull() & data1['sales'].isnull()]['date']
print(null_count)

154017     2021-05-23
154099     2021-05-21
154149     2021-05-24
154155     2021-05-22
429815     2021-12-10
429860     2021-12-09
733468     2021-06-27
734198     2021-06-26
738594     2021-05-29
738600     2021-05-23
738632     2021-05-31
738769     2021-05-22
738805     2021-05-30
738816     2021-05-21
738994     2021-05-24
1028203    2021-05-22
1028500    2021-05-23
1028566    2021-05-24
1028753    2021-05-21
1231189    2021-05-21
1231497    2021-05-23
1231500    2021-05-22
1231580    2021-05-29
1231636    2021-05-31
1231739    2021-05-24
1231755    2021-05-30
1451592    2021-06-20
1451661    2021-06-12
1451854    2021-07-04
1451892    2021-06-27
1452131    2021-06-19
1452275    2021-06-13
1452400    2021-07-11
1452632    2021-06-26
3464803    2021-05-23
3464921    2021-05-31
3464993    2021-05-21
3465006    2021-05-24
3465126    2021-05-30
3465135    2021-05-22
3465197    2021-05-29
3744049    2021-07-11
3860150    2021-06-12
3860232    2021-06-13
3862121    2021-12-09
3862202   

In [5]:
pd.set_option('display.max_rows', None)
matching_rows = data3[data3['date'].isin(null_count)][['date', 'holiday', 'warehouse']]
print(matching_rows)

             date  holiday    warehouse
230    2021-06-13        0     Munich_1
470    2021-07-04        0     Prague_1
776    2021-06-19        0   Budapest_1
1079   2021-12-09        0       Brno_1
1091   2021-12-10        0     Prague_2
1118   2021-07-04        0       Brno_1
1255   2021-06-26        0     Prague_3
1568   2021-05-22        0     Prague_1
1740   2021-05-31        0     Munich_1
2315   2021-07-04        0     Prague_2
2422   2021-12-09        0     Prague_2
2432   2021-07-11        0       Brno_1
2497   2021-06-27        0  Frankfurt_1
2548   2021-06-27        0     Prague_3
2553   2021-06-20        0     Munich_1
2561   2021-06-12        0   Budapest_1
3807   2021-06-20        0  Frankfurt_1
4733   2021-05-21        0  Frankfurt_1
5063   2021-05-24        1   Budapest_1
5120   2021-05-30        0     Prague_1
5126   2021-05-29        0       Brno_1
5182   2021-06-19        0     Prague_2
5221   2021-05-30        0   Budapest_1
5382   2021-05-31        0  Frankfurt_1


In [6]:
for date in null_count:
    data1.loc[data1['date'] == date, 'total_orders'] = data1.loc[
    data1['date'] <= date, 'total_orders'].rolling(window = 7, min_periods = 1).mean().iloc[-1]
    data1.loc[data1['date'] == date, 'sales'] = data1.loc[
    data1['date'] <= date, 'sales'].rolling(window = 7, min_periods = 1).mean().iloc[-1]
print(data1.isnull().sum())
print(data1.shape)

unique_id          0
date               0
warehouse          0
total_orders       0
sales              0
sell_price_main    0
availability       0
type_0_discount    0
type_1_discount    0
type_2_discount    0
type_3_discount    0
type_4_discount    0
type_5_discount    0
type_6_discount    0
dtype: int64
(4007419, 14)


In [7]:
print(data2.info())
print(data2.isnull().sum()) #info of test set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47021 entries, 0 to 47020
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   unique_id        47021 non-null  int64  
 1   date             47021 non-null  object 
 2   warehouse        47021 non-null  object 
 3   total_orders     47021 non-null  float64
 4   sell_price_main  47021 non-null  float64
 5   type_0_discount  47021 non-null  float64
 6   type_1_discount  47021 non-null  float64
 7   type_2_discount  47021 non-null  float64
 8   type_3_discount  47021 non-null  float64
 9   type_4_discount  47021 non-null  float64
 10  type_5_discount  47021 non-null  float64
 11  type_6_discount  47021 non-null  float64
dtypes: float64(9), int64(1), object(2)
memory usage: 4.3+ MB
None
unique_id          0
date               0
warehouse          0
total_orders       0
sell_price_main    0
type_0_discount    0
type_1_discount    0
type_2_discount    0
type_3_disco

In [8]:
data2.date.value_counts().sort_index()

date
2024-06-03    3386
2024-06-04    3417
2024-06-05    3450
2024-06-06    3439
2024-06-07    3419
2024-06-08    3362
2024-06-09    3294
2024-06-10    3315
2024-06-11    3323
2024-06-12    3360
2024-06-13    3349
2024-06-14    3353
2024-06-15    3309
2024-06-16    3245
Name: count, dtype: int64

In [9]:
print(data3.info())
print(data3.isnull().sum()) #info of calender set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23016 entries, 0 to 23015
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   date                    23016 non-null  object
 1   holiday_name            930 non-null    object
 2   holiday                 23016 non-null  int64 
 3   shops_closed            23016 non-null  int64 
 4   winter_school_holidays  23016 non-null  int64 
 5   school_holidays         23016 non-null  int64 
 6   warehouse               23016 non-null  object
dtypes: int64(4), object(3)
memory usage: 1.2+ MB
None
date                          0
holiday_name              22086
holiday                       0
shops_closed                  0
winter_school_holidays        0
school_holidays               0
warehouse                     0
dtype: int64


In [10]:
print(data4.info())
print(data4.isnull().sum()) #info of inventory set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5432 entries, 0 to 5431
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   unique_id            5432 non-null   int64 
 1   product_unique_id    5432 non-null   int64 
 2   name                 5432 non-null   object
 3   L1_category_name_en  5432 non-null   object
 4   L2_category_name_en  5432 non-null   object
 5   L3_category_name_en  5432 non-null   object
 6   L4_category_name_en  5432 non-null   object
 7   warehouse            5432 non-null   object
dtypes: int64(2), object(6)
memory usage: 339.6+ KB
None
unique_id              0
product_unique_id      0
name                   0
L1_category_name_en    0
L2_category_name_en    0
L3_category_name_en    0
L4_category_name_en    0
warehouse              0
dtype: int64


In [11]:
print(data5.info())
print(data5.isnull().sum()) #info of solution set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47021 entries, 0 to 47020
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         47021 non-null  object
 1   sales_hat  47021 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 734.8+ KB
None
id           0
sales_hat    0
dtype: int64


In [12]:
print(data6.info())
print(data6.isnull().sum()) #info of test weights set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5390 entries, 0 to 5389
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   unique_id  5390 non-null   int64  
 1   weight     5390 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 84.3 KB
None
unique_id    0
weight       0
dtype: int64


In [13]:
merge1 = pd.merge(data1, data6, on = 'unique_id', how = 'left')
print(merge1.shape)

(4007419, 15)


In [14]:
merge2 = pd.merge(merge1, data3, on = ['date', 'warehouse'], how = 'left')
print(merge2.shape)

(4007419, 20)


In [15]:
merge3 = pd.merge(merge2, data4, on = ['unique_id', 'warehouse'], how = 'left')
print(merge3.shape)

(4007419, 26)


In [16]:
merge3['date'] = pd.to_datetime(merge3['date'], errors = 'coerce')
merge3['day'] = merge3['date'].dt.day
merge3['month'] = merge3['date'].dt.month
merge3['year'] = merge3['date'].dt.year
print(merge3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4007419 entries, 0 to 4007418
Data columns (total 29 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   unique_id               int64         
 1   date                    datetime64[ns]
 2   warehouse               object        
 3   total_orders            float64       
 4   sales                   float64       
 5   sell_price_main         float64       
 6   availability            float64       
 7   type_0_discount         float64       
 8   type_1_discount         float64       
 9   type_2_discount         float64       
 10  type_3_discount         float64       
 11  type_4_discount         float64       
 12  type_5_discount         float64       
 13  type_6_discount         float64       
 14  weight                  float64       
 15  holiday_name            object        
 16  holiday                 int64         
 17  shops_closed            int64         
 18  wi

In [17]:
merge4 = pd.merge(data2, data3, on = ['date', 'warehouse'], how = 'left')
print(merge4.shape)

(47021, 17)


In [18]:
merge5 = pd.merge(merge4, data4, on = ['unique_id', 'warehouse'], how = 'left')
print(merge5.shape)

(47021, 23)


In [19]:
merge5.insert(4, 'sales', [None] * len(merge5))
print("Column created successfully")

Column created successfully


In [20]:
merge5.insert(6, 'availability', [None] * len(merge5))
print("Column created successfully")

Column created successfully


In [21]:
merge5.insert(14, 'weight', [None] * len(merge5))
print("Column created successfully")

Column created successfully


In [22]:
merge5['date'] = pd.to_datetime(merge5['date'], errors = 'coerce')
merge5['day'] = merge5['date'].dt.day
merge5['month'] = merge5['date'].dt.month
merge5['year'] = merge5['date'].dt.year
print(merge5.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47021 entries, 0 to 47020
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   unique_id               47021 non-null  int64         
 1   date                    47021 non-null  datetime64[ns]
 2   warehouse               47021 non-null  object        
 3   total_orders            47021 non-null  float64       
 4   sales                   0 non-null      object        
 5   sell_price_main         47021 non-null  float64       
 6   availability            0 non-null      object        
 7   type_0_discount         47021 non-null  float64       
 8   type_1_discount         47021 non-null  float64       
 9   type_2_discount         47021 non-null  float64       
 10  type_3_discount         47021 non-null  float64       
 11  type_4_discount         47021 non-null  float64       
 12  type_5_discount         47021 non-null  float6

In [23]:
print(merge5.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47021 entries, 0 to 47020
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   unique_id               47021 non-null  int64         
 1   date                    47021 non-null  datetime64[ns]
 2   warehouse               47021 non-null  object        
 3   total_orders            47021 non-null  float64       
 4   sales                   0 non-null      object        
 5   sell_price_main         47021 non-null  float64       
 6   availability            0 non-null      object        
 7   type_0_discount         47021 non-null  float64       
 8   type_1_discount         47021 non-null  float64       
 9   type_2_discount         47021 non-null  float64       
 10  type_3_discount         47021 non-null  float64       
 11  type_4_discount         47021 non-null  float64       
 12  type_5_discount         47021 non-null  float6

In [24]:
merge3['product_unique_id'].nunique()

2653

In [25]:
merge3['log_sales'] = np.log(merge3['sales'] + 1)
merge3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4007419 entries, 0 to 4007418
Data columns (total 30 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   unique_id               int64         
 1   date                    datetime64[ns]
 2   warehouse               object        
 3   total_orders            float64       
 4   sales                   float64       
 5   sell_price_main         float64       
 6   availability            float64       
 7   type_0_discount         float64       
 8   type_1_discount         float64       
 9   type_2_discount         float64       
 10  type_3_discount         float64       
 11  type_4_discount         float64       
 12  type_5_discount         float64       
 13  type_6_discount         float64       
 14  weight                  float64       
 15  holiday_name            object        
 16  holiday                 int64         
 17  shops_closed            int64         
 18  wi

In [26]:
merge3 = merge3.sort_values('date')
holiday_dates = merge3[merge3['holiday'] == 1][['date']].rename(columns={'date': 'holiday_date'})

merge3 = pd.merge_asof(merge3, holiday_dates, left_on = 'date', right_on = 'holiday_date', direction = 'backward')
merge3.rename(columns = {'holiday_date': 'last_holiday_date'}, inplace = True)

merge3 = pd.merge_asof(merge3, holiday_dates, left_on = 'date', right_on = 'holiday_date', direction = 'forward')
merge3.rename(columns = {'holiday_date': 'next_holiday_date'}, inplace = True)

merge3['since_last_holiday'] = (merge3['date'] - merge3['last_holiday_date']).dt.days
merge3['until_next_holiday'] = (merge3['next_holiday_date'] - merge3['date']).dt.days

In [27]:
def make_time_features(merge3):
    merge3['date'] = pd.to_datetime(merge3['date'], errors = 'coerce')    
    
    merge3['day'] = merge3['date'].dt.dayofweek.astype(int)
    merge3['week'] = merge3['date'].dt.isocalendar().week.astype(int)
    merge3['month'] = merge3['date'].dt.month.astype(int)
    merge3['day_of_week'] = merge3['date'].dt.strftime('%A')
    merge3['day_in_month'] = merge3['date'].dt.day.astype(int)
    merge3['day_of_year'] = merge3['date'].dt.dayofyear.astype(int)
    merge3['year'] = merge3['date'].dt.year.astype(int)

    merge3['sin_month'] = np.sin(2 * np.pi * merge3['month'] / 12)
    merge3['sin_day'] = np.sin(2 * np.pi * merge3['day'] / 7)
    merge3['sin_week'] = np.sin(2 * np.pi * merge3['week'] / 53)
    merge3['sin_day_year'] = np.sin(2 * np.pi * merge3['day_of_year'] / 365)

    min_date = merge3['date'].min()
    merge3['days_since_start'] = merge3['date'].apply(lambda x: (x - min_date).days)

    merge3['lag_1'] = merge3['log_sales'].shift(1)
    merge3['lag_7'] = merge3['log_sales'].shift(7)
    merge3['lag_30'] = merge3['log_sales'].shift(30)

    merge3['rolling_mean_7'] = merge3['log_sales'].rolling(window = 7).mean()
    merge3['rolling_std_7'] = merge3['log_sales'].rolling(window = 7).std()
    merge3['rolling_mean_30'] = merge3['log_sales'].rolling(window = 30).mean()
    merge3['rolling_std_30'] = merge3['log_sales'].rolling(window = 30).std()

    merge3['rolling_min_7'] = merge3['log_sales'].rolling(window = 7).min()
    merge3['rolling_max_7'] = merge3['log_sales'].rolling(window = 7).max()
    merge3['rolling_min_30'] = merge3['log_sales'].rolling(window = 30).min()
    merge3['rolling_max_30'] = merge3['log_sales'].rolling(window = 30).max()

    merge3['rolling_sum_7'] = merge3['log_sales'].rolling(window = 7).sum()
    merge3['rolling_sum_30'] = merge3['log_sales'].rolling(window = 30).sum()

    merge3['expanding_mean'] = merge3['log_sales'].expanding().mean()

    merge3['is_holiday'] = merge3['holiday'].apply(lambda x: 1 if x else 0)
    merge3['is_weekday'] = merge3['day_of_week'].apply(lambda x: 1 if x in [0, 1, 2, 3, 4] else 0)
    merge3['is_weekend'] = merge3['day_of_week'].apply(lambda x: 1 if x in [5, 6] else 0)

    merge3 = pd.get_dummies(merge3, columns = ['day_of_week'], drop_first = True)
    merge3 = merge3.replace([np.inf, -np.inf], np.nan).fillna(0)

    return merge3

merge3 = make_time_features(merge3)

In [28]:
def make_time_features(merge5):
    merge5['date'] = pd.to_datetime(merge5['date'], errors = 'coerce')    
    
    merge5['day'] = merge5['date'].dt.dayofweek.astype(int)
    merge5['week'] = merge5['date'].dt.isocalendar().week.astype(int)
    merge5['month'] = merge5['date'].dt.month.astype(int)
    merge5['day_of_week'] = merge5['date'].dt.strftime('%A')
    merge5['day_in_month'] = merge5['date'].dt.day.astype(int)
    merge5['day_of_year'] = merge5['date'].dt.dayofyear.astype(int)
    merge5['year'] = merge5['date'].dt.year.astype(int)

    merge5['sin_month'] = np.sin(2 * np.pi * merge5['month'] / 12)
    merge5['sin_day'] = np.sin(2 * np.pi * merge5['day'] / 7)
    merge5['sin_week'] = np.sin(2 * np.pi * merge5['week'] / 53)
    merge5['sin_day_year'] = np.sin(2 * np.pi * merge5['day_of_year'] / 365)

    min_date = merge5['date'].min()
    merge5['days_since_start'] = merge5['date'].apply(lambda x: (x - min_date).days)

    merge5['is_holiday'] = merge5['holiday'].apply(lambda x: 1 if x else 0)
    merge5['is_weekday'] = merge5['day_of_week'].apply(lambda x: 1 if x in [0, 1, 2, 3, 4] else 0)
    merge5['is_weekend'] = merge5['day_of_week'].apply(lambda x: 1 if x in [5, 6] else 0)

    merge5 = pd.get_dummies(merge5, columns = ['day_of_week'], drop_first = True)
    merge5 = merge5.replace([np.inf, -np.inf], np.nan).fillna(0)

    return merge5

merge5 = make_time_features(merge5)

C:\Users\HP\AppData\Local\Temp\ipykernel_9708\1024312215.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merge5 = merge5.replace([np.inf, -np.inf], np.nan).fillna(0)


In [29]:
merge3 = merge3.sort_values(by = ["product_unique_id", "year", "sin_day_year"]).reset_index(drop = True)
print("Procedure successfull")

Procedure successfull


In [30]:
def assign_around_hol(merge3, original=1):
    merge3["holiday_name"] = merge3["holiday_name"].fillna("")
    for warehouse in merge3["warehouse"].unique():
        warehouse_merge3 = merge3[merge3["warehouse"] == warehouse].copy()

        warehouse_merge3["holiday"] = -2 * warehouse_merge3["holiday"].shift(-1).fillna(0) + warehouse_merge3["holiday"] + 2 * warehouse_merge3["holiday"].shift(1).fillna(0)

        warehouse_merge3["holiday_name"] = warehouse_merge3["holiday_name"].replace("", np.nan)

        warehouse_merge3["holiday_name"] = warehouse_merge3["holiday_name"].astype("category")

        warehouse_merge3["holiday_name"] = warehouse_merge3["holiday_name"].ffill()
        warehouse_merge3["holiday_name"] = warehouse_merge3["holiday_name"].bfill()

        merge3.loc[merge3["warehouse"] == warehouse, "holiday_name"] = warehouse_merge3["holiday_name"]

assign_around_hol(merge3)
assign_around_hol(merge5)

In [31]:
merge3 = merge3.drop(['date'], axis=1)
merge5 = merge5.drop(['date'], axis=1)
print("Column Dropped successfully")

Column Dropped successfully


In [32]:
columns = ["holiday", "shops_closed", 'winter_school_holidays', 'school_holidays']

def make_lag(merge3, original, columns):
    for column in columns:
        for lag in original:
            merge3[f"{column}_{lag}"] = np.nan  
            
    for column in columns:
        for warehouse in merge3["warehouse"].unique():
            warehouse_data = merge3[merge3["warehouse"] == warehouse].copy()
            
            for lag in original:
                if lag == 0:
                    continue
                warehouse_data[f"{column}_{lag}"] = warehouse_data[column].shift(lag)

            merge3.loc[merge3["warehouse"] == warehouse, [f"{column}_{lag}" for lag in original]] = warehouse_data[[f"{column}_{lag}" for lag in original]]   
    return merge3

In [33]:
y = merge3.log_sales
features = ['sin_month', 'sin_day', 'sin_week', 'sin_day_year', 'lag_1',
'since_last_holiday', 'lag_7', 'lag_30', 'rolling_mean_7', 'rolling_mean_30', 'rolling_std_7',
'until_next_holiday', 'rolling_std_30', 'rolling_min_7', 'rolling_max_7', 'rolling_min_30', 'rolling_max_30',
'rolling_sum_7', 'rolling_sum_30', 'expanding_mean', 'is_holiday', 'is_weekday', 'is_weekend',
'total_orders', 'sell_price_main', 'availability']
X = merge3[features]
X = pd.get_dummies(X)
scaler = StandardScaler()
X[['sin_month', 'sin_day', 'sin_week', 'sin_day_year', 'lag_1',
'since_last_holiday', 'lag_7', 'lag_30', 'rolling_mean_7', 'rolling_mean_30', 'rolling_std_7',
'until_next_holiday', 'rolling_std_30', 'rolling_min_7', 'rolling_max_7', 'rolling_min_30', 'rolling_max_30',
'rolling_sum_7', 'rolling_sum_30', 'expanding_mean', 'is_holiday', 'is_weekday', 'is_weekend',
'total_orders', 'sell_price_main', 'availability']] = scaler.fit_transform(X[['sin_month', 'sin_day', 'sin_week', 'sin_day_year', 'lag_1',
'since_last_holiday', 'lag_7', 'lag_30', 'rolling_mean_7', 'rolling_mean_30', 'rolling_std_7',
'until_next_holiday', 'rolling_std_30', 'rolling_min_7', 'rolling_max_7', 'rolling_min_30', 'rolling_max_30',
'rolling_sum_7', 'rolling_sum_30', 'expanding_mean', 'is_holiday', 'is_weekday', 'is_weekend',
'total_orders', 'sell_price_main', 'availability']])
print(X.isnull().sum())
print(y.isnull().sum())

sin_month             0
sin_day               0
sin_week              0
sin_day_year          0
lag_1                 0
since_last_holiday    0
lag_7                 0
lag_30                0
rolling_mean_7        0
rolling_mean_30       0
rolling_std_7         0
until_next_holiday    0
rolling_std_30        0
rolling_min_7         0
rolling_max_7         0
rolling_min_30        0
rolling_max_30        0
rolling_sum_7         0
rolling_sum_30        0
expanding_mean        0
is_holiday            0
is_weekday            0
is_weekend            0
total_orders          0
sell_price_main       0
availability          0
dtype: int64
0


In [34]:
print(merge3.get('day_of_week'))
print(merge5.get('day_of_week'))

None
None


In [35]:
drop_columns = ['warehouse', 'holiday_name']
merge3[drop_columns] = merge3[drop_columns].astype(str)
merge5[drop_columns] = merge5[drop_columns].astype(str)

encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value = np.nan)
merge3[drop_columns] = encoder.fit_transform(merge3[drop_columns])
merge5[drop_columns] = encoder.transform(merge5[drop_columns])
warehouse_decoder = {i: encoder.categories_[0][i] for i in range(len(encoder.categories_[0]))}
print(warehouse_decoder)

merge3['warehouse'] = merge3['warehouse'].astype("category")
merge5['warehouse'] = merge5['warehouse'].astype("category")

merge3['warehouse'] = merge3['warehouse'].cat.add_categories('Unknown')
merge5['warehouse'] = merge5['warehouse'].cat.add_categories('Unknown')
merge5 = merge5.fillna({'warehouse': 'Unknown', 'holiday_name': 'Unknown'})

print("Encoding Successful")

{0: 'Brno_1', 1: 'Budapest_1', 2: 'Frankfurt_1', 3: 'Munich_1', 4: 'Prague_1', 5: 'Prague_2', 6: 'Prague_3'}
Encoding Successful


In [37]:
tscv = TimeSeriesSplit(n_splits=10)
for train_idx, val_idx in tscv.split(X):
    train_X, val_X = X.iloc[train_idx], X.iloc[val_idx]
    train_y, val_y = y.iloc[train_idx], y.iloc[val_idx]

In [ ]:
sarima_model = SARIMAX(train_y, order = (1,1,1), seasonal_order = (1,1,1,12))
sarima_fit = sarima_model.fit()

sarima_train_pred = sarima_fit.fittedvalues
sarima_residuals = train_y - sarima_train_pred

rf_model = xgb.XGBRegressor(
    n_estimators = 100,      
    learning_rate = 0.02,     
    max_depth = 3,            
    subsample = 0.7,          
    colsample_bytree = 0.7,   
    gamma = 0.1,              
    min_child_weight = 1,     
    scale_pos_weight = 1,     
    missing = -999,           
    random_state = 1111,      
    tree_method = 'hist',     
    n_jobs = -1)

rf_model.fit(
    train_X, train_y, 
    eval_set=[(val_X, val_y)], 
    early_stopping_rounds=50,
    verbose=1)

rf_model.fit(train_X, sarima_residuals, eval_set = [(val_X, val_y - sarima_fit.predict(start = len(train_y), end = len(train_y) + len(val_y) - 1))], verbose = 1)
sarima_val_pred = sarima_fit.predict(start = len(train_y), end = len(train_y) + len(val_y) - 1)
xgb_residuals = rf_model.predict(val_X)

final_forecast = sarima_val_pred + xgb_residuals

rf_val_mae = mean_absolute_error(final_forecast, val_y)
print("Validation MAE for SARIMA + XGBoost Model: {:,.0f}".format(rf_val_mae))
print("Sample Predictions:", final_forecast[:5])
print("Actual Values:", val_y[:5].values)

In [ ]:
explainer = shap.Explainer(rf_model, train_X)
shap_values = explainer(train_X)
shap.summary_plot(shap_values, train_X)

In [ ]:
print('The accuracy of the model is: ', rf_model.score(val_X, val_y)) 
print('The accuracy of the training model is: ', rf_model.score(train_X, train_y))

In [ ]:
sarima_model = SARIMAX(y, order = (1,1,1), seasonal_order = (1,1,1,12))
sarima_fit = sarima_model.fit()

sarima_train_pred = sarima_fit.fittedvalues
sarima_residuals = y - sarima_train_pred

features = [col for col in features if col in X.columns and col in merge5.columns]
train_X = pd.get_dummies(X[features])
test_X = pd.get_dummies(merge5[features])
test_X = test_X.reindex(columns = train_X.columns, fill_value=0)
rf_model_on_full_data = xgb.XGBRegressor(
    n_estimators = 100,      
    learning_rate = 0.02,     
    max_depth = 3,            
    subsample = 0.7,          
    colsample_bytree = 0.7,   
    gamma = 0.1,              
    min_child_weight = 1,     
    scale_pos_weight = 1,     
    missing = -999,           
    random_state = 1111,      
    tree_method = 'hist',     
    n_jobs = -1  
    )

rf_model_on_full_data.fit(train_X, sarima_residuals)
sarima_test_pred = sarima_fit.predict(start = len(y), end = len(y) + len(test_X) - 1)

xgb_residuals = rf_model_on_full_data.predict(test_X)
final_test_preds = sarima_test_pred + xgb_residuals
print("Sample Predictions:", final_test_preds[:5])

In [ ]:
test_preds = sarima_test_pred + xgb_residuals
test_preds = np.abs(test_preds)

In [ ]:
data5['id'] = data5['id']
print("Check the 'id' column:")
print(data5['id'].head(5))
output = pd.DataFrame({'id': data5.id,
                       'sales_hat': test_preds})
output.to_csv('submission.csv', index=False)

In [ ]:
min_length = min(len(val_y), len(final_test_preds))
mae = mean_absolute_error(val_y[:min_length], final_test_preds[:min_length])
mse = mean_squared_error(val_y[:min_length], final_test_preds[:min_length])
rmse = np.sqrt(mse)
r2 = r2_score(val_y[:min_length], final_test_preds[:min_length])

print(f"R²: {r2}")
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")

In [ ]:
threshold = 0.5
min_length = min(len(val_y), len(final_test_preds))
y_pred_class = np.where(final_test_preds[:min_length] >= threshold, 1, 0)
val_y_class = np.where(val_y[:min_length] >= threshold, 1, 0)

accuracy = accuracy_score(val_y_class, y_pred_class)
f1 = f1_score(val_y_class, y_pred_class, average='weighted')
precision = precision_score(val_y_class, y_pred_class, average='weighted')
recall = recall_score(val_y_class, y_pred_class, average='weighted')
cm = confusion_matrix(val_y_class, y_pred_class)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Confusion Matrix:\n{cm}")

In [ ]:
threshold = 0.5
min_length = min(len(val_y), len(final_test_preds))
y_pred_class = np.where(final_test_preds[:min_length] >= threshold, 1, 0)
val_y_class = np.where(val_y[:min_length] >= threshold, 1, 0)
report = classification_report(val_y_class, y_pred_class)
print("Classification Report:\n", report)

In [ ]:
feature_importance = rf_model.get_booster().get_score(importance_type='gain')
importances_df = pd.DataFrame(feature_importance.items(), columns=['Feature', 'Importance'])
importances_df = importances_df.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(importances_df['Feature'][:15], importances_df['Importance'][:15], color='royalblue')
plt.gca().invert_yaxis()  # Flip y-axis for better visualization
plt.xlabel("Feature Importance (Gain)")
plt.title("Top 15 Important Features in XGBoost")
plt.show()